# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Camila Monter, cm755

**ID**:
5244001

> **Due Date**
>
> Wednesday, 10/16/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [155]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/lab-3-CamilaMonter`


In [156]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

`JuMP` requires the loading of a solver. \[Each supported solver works
for certain classes of problems, and some are open source while others
require a commercial license\]. We will use the `HiGHS` solver, which is
open source and works for linear, mixed integer linear, and quadratic
programs.

In this lab we will walk through the steps involved in coding a linear
program in HiGHS, solving it, and querying the solution.

## Exercise (3 points)

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
$\$0.12$ per bf to purchase and fir costs $\$0.08$ per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board, which yields no
revenue for the mill.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

First, we need to identify our decision variables. While there are
several options, we will use $G_i$, the amount of each grade the mill
produces (in \$/1000 bf).

Using these decision variables, formulate a linear program to maximize
the profit of the mill subject to the supply constraints on spruce and
fir.

> **JuMP Syntax**
>
> The core pieces of setting up a `JuMP` model involve specifying the
> model and adding variables, the objective, and constraints. At the
> most simple level, this syntax looks like this:
>
> ``` julia
> m = Model(HiGHS.Optimizer)
> @variable(m, lb <= x <= ub) # if you do not have upper or lower bounds, you can drop those accordingly
> @variable(m, lb <= y <= ub)
> @objective(m, Max, 100x + 250y) # replace Max with Min depending on the problem
> @constraint(m, label, 6x + 12y <= 80) # replace "label" with some meaningful string you would like to use later to query shadow prices, or drop it
> ```
>
> You can add more constraints or more variables as needed.

> **Using Array Syntax**
>
> You can set up multiple variables or constraints at once using array
> syntax. For example, the following are equivalent:
>
> ``` julia
> @variable(m, G1 >= 0)
> @variable(m, G2 >= 0)
> @variable(m, G3 >= 0)
> ```
>
> and
>
> ``` julia
> @variable(m, G[1:3] >= 0)
> ```
>
> You can also set up multiple constraints using arrays of coefficients
> and/or bounds. For example:
>
> ``` julia
> I = 1:3
> d = [0; 3; 5]
> @constraint(m, demand[i in I], G[i] >= d[i])
> ```

`JuMP` is finicky about changing objects and constraints, so I recommend
setting up all of the model syntax in one notebook cell, which is what
we will do here.

**Derivation**

Using the JuMP Syntax, I set up a model to optimize lumber production, defining variables using array syntax. I also set variables for the cost of spruce and fir, as well as the revenue per plywood grade. This allows for neater and easier to follow code.

Setting the objective to maximize profit, I devised the governing equation based on Table 1, multiplying the plywood grade by it's associated revenue. This is the gross revenue. To obtain profit, we have to subtract by the cost of each type of wood and the amount of inputs they require.

The two constraints were written based on the total maximum bf of forest available for each type of wood, and the inputs of the wood needed to make each level of plywood.

Profit can be defined as -- Profit = Total Revenue - Total Cost
Total revenue from each grade of plywood produced can be defined as:

Revenue from grade 1: 
$$ G_1 \times 400 \, \text{(in dollars per 1000 bf)} $$

Revenue from grade 2: 
$$ G_2 \times 520 \, \text{(in dollars per 1000 bf)} $$

Revenue from grade 3: 
$$ G_3 \times 700 \, \text{(in dollars per 1000 bf)} $$

Thus,

Total Revenue = $$ G_1 \cdot 400 + G_2 \cdot 520 + G_3 \cdot 700 $$

Cost of fir:
For grade 1: 
$$ 1.5 \cdot G_1 \cdot C_f $$

For grade 2: 
$$ 2.0 \cdot G_2 \cdot C_f $$

For grade 3: 
$$ 2.0 \cdot G_3 \cdot C_f $$

Thus, the total cost for fir is:

$$ \text{Cost of Fir} = (1.5 \cdot G_1 + 2.0 \cdot G_2 + 2.0 \cdot G_3) \cdot C_f $$

Putting this together, the profit equation becomes:

$$ 
\text{Profit} = \left( G_1 \cdot 400 + G_2 \cdot 520 + G_3 \cdot 700 \right) - \left( \left( 0.5 \cdot G_1 + 1.0 \cdot G_2 + 1.5 \cdot G_3 \right) \cdot C_s + \left( 1.5 \cdot G_1 + 2.0 \cdot G_2 + 2.0 \cdot G_3 \right) \cdot C_f \right) 
$$

Inserting the costs defined at the beginning of the code:

$$ 
\text{Objective Function} = \left( G_1 \cdot 400 + G_2 \cdot 520 + G_3 \cdot 700 \right) - \left( 0.5 \cdot G_1 + 1.0 \cdot G_2 + 1.5 \cdot G_3 \right) \cdot 120 - \left( 1.5 \cdot G_1 + 2.0 \cdot G_2 + 2.0 \cdot G_3 \right) \cdot 80 
$$

**Constraints Derivation**

Spruce Constraint

This constraint ensures that the amount of spruce used does not exceed the available supply of 320,000 bf. Each grade of plywood requires a certain amount of spruce:

Grade 1: 0.5 bf of spruce per bf of plywood

Grade 2: 1.0 bf of spruce per bf of plywood

Grade 3: 1.5 bf of spruce per bf of plywood

$$ 0.5 \cdot G_1 + 1.0 \cdot G_2 + 1.5 \cdot G_3 \leq 320 $$

This constraint ensures that the total spruce required for producing the plywood does not exceed the supply.

Fir Constraint

Similarly, the fir constraint ensures that the amount of fir used does not exceed the available supply of 720,000 bf. Each grade of plywood requires:

Grade 1: 1.5 bf of fir per bf of plywood

Grade 2: 2.0 bf of fir per bf of plywood

Grade 3: 2.0 bf of fir per bf of plywood

Thus, the constraint can be expressed as:
$$ 1.5 \cdot G_1 + 2.0 \cdot G_2 + 2.0 \cdot G_3 \leq 720 $$

This constraint ensures that the total fir required for producing the plywood does not exceed the supply.


In [157]:
forest_model = Model(HiGHS.Optimizer) # initialize model object
#Define variables

#Cost of wood ($/1000 bf)
Cs=0.12*1000
Cf=0.08*1000

#Revenue (#$/1000)
RevP1 = 400
RevP2 = 520 
RevP3 = 700 

# non-negativity constraints
@variable(forest_model, G1 >= 0)
@variable(forest_model, G2 >= 0)
@variable(forest_model, G3 >= 0)

# uncomment the following lines and add the objective and constraints as needed for the model
@objective(forest_model, Max, (G1*RevP1 + G2*RevP2 + G3*RevP3) - (0.5*G1 + 1.0*G2 + 1.5*G3)*Cs - (1.5*G1 + 2.0*G2 + 2.0*G3)*Cf) 
# Spruce constraint
@constraint(forest_model, spruce_limit, 0.5*G1 + 1.0*G2 + 1.5*G3 <= 320)
# Fir constraint
@constraint(forest_model, fir_limit, 1.5*G1 + 2.0*G2 + 2.0*G3 <= 720)
print(forest_model) # this outputs a nicely formatted summary of the model so you can check your specification

Max 220 G1 + 240 G2 + 360 G3
Subject to
 spruce_limit : 0.5 G1 + G2 + 1.5 G3 ≤ 320
 fir_limit : 1.5 G1 + 2 G2 + 2 G3 ≤ 720
 G1 ≥ 0
 G2 ≥ 0
 G3 ≥ 0


Next, to optimize, use the `optimize!()` function:

In [158]:
optimize!(forest_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-01, 2e+00]
  Cost   [2e+02, 4e+02]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 7e+02]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.1999929744e+02 Ph1: 2(8.5); Du: 3(819.999) 0s
          2     1.1200000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.00


You should get confirmation that a solution was found; if one was not,
there’s a chance something was wrong with your model formulation.

To find the values of the decision variables, use `value()` (which can
be broadcasted over variable arrays):

In [159]:
@show value.(G1);
@show value.(G2);
@show value.(G3);

value.(G1) = 352.0
value.(G2) = 0.0
value.(G3) = 96.0


**Solution**

Printing our values, we obtain the following solution:

352,000 bf of grade 1

0 bf of grade 2

96,000 bf of grade 3

Similarly, `objective_value()` finds the optimal value of the objective:`

In [160]:
@show objective_value(forest_model)

objective_value(forest_model) = 112000.0


112000.0

**Solution**

The optimal total profit from producing three grades of plywood is $112,000

Finally, we can find the dual values of the constraints with
`shadow_price()`. Do this for the constraints in your model using the
block below.

In [161]:
@show shadow_price(spruce_limit);
@show shadow_price(fir_limit);

shadow_price(spruce_limit) = 80.0
shadow_price(fir_limit) = 120.0


**Solution**
For the spruce constraint, relaxing the constraint by one additional unit of wood would increase the mill's profit by $80.

For the fir constraint, relaxing the constraint by one additional unit of wood would increase the mill's profit by $120.



`JuMP` also lets you evaluate other expressions that you might be
interested in based on the solutions. For example, you can use the
following block to calculate the total amount of plywood the mill would
produce under the optimal solution:

In [162]:
@expression(forest_model, total_plywood, G1 + G2 + G3)
@show value.(total_plywood);

value.(total_plywood) = 448.0


**Solution**
Under the optimal solution, the total amount of plywood produced by the mill is 440,000 bf

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

Worked with Priya Shah and Bailey Belinger